In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
sns.set()
# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery
from google.colab import drive
drive.mount('/content/gdrive')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
auth.authenticate_user()
print('Authenticated')


Authenticated


In [0]:
project_id='eicudata'

# Extracting features from the eICU dataset for Mortality and los prediction

In [0]:
%%bigquery --project $project_id rawdf
select b.patientunitstayid,unitvisitnumber, gender,age,ethnicity,apacheadmissiondx,admissionheight,admissionweight, eyes, motor, verbal, (eyes+motor+verbal) as GCSTotal, temperature,respiratoryrate,heartrate,glucose,(select min(labresult)
from `physionet-data.eicu_crd.lab`
where patientunitstayid = a.patientunitstayid and labname like '%O2 Sat%'
group by labname) as O2,fio2,creatinine,ph,hospitaldischargestatus,unitdischargestatus,(unitdischargeoffset-hospitaladmitoffset)/60/24 as ICUstay
from `physionet-data.eicu_crd.apacheapsvar` a
join `physionet-data.eicu_crd.patient` b ON (
a.patientunitstayid =b.patientunitstayid
)
order by b.patientunitstayid;

In [0]:
%%bigquery --project $project_id vital
select patientunitstayid, avg(systemicsystolic) as SystolicBP, avg(systemicdiastolic) as DiastolicBP, avg((2*systemicdiastolic + systemicsystolic)/3) as MAP
from `physionet-data.eicu_crd.vitalperiodic` 
where systemicsystolic & systemicdiastolic is not null
group by patientunitstayid
order by patientunitstayid

In [43]:
vital

,patientunitstayid,SystolicBP,DiastolicBP,MAP
0,141168,92.707317,51.000000,64.902439
1,141194,92.828488,50.636628,64.700581
2,141227,123.072581,78.064516,93.067204
3,141233,103.366634,55.650501,71.555879
4,141244,132.334862,57.155963,82.215596
...,...,...,...,...
46655,3353194,115.167622,61.166189,79.166667
46656,3353197,122.685241,49.463950,73.871047
46657,3353199,151.571674,41.611159,78.264664
46658,3353216,134.452465,77.519366,96.497066


In [0]:
rawdf1 = pd.merge(left=rawdf,right=vital,how='left')

In [49]:
rawdf1

,patientunitstayid,unitvisitnumber,gender,age,ethnicity,apacheadmissiondx,admissionheight,admissionweight,eyes,motor,verbal,GCSTotal,temperature,respiratoryrate,heartrate,glucose,O2,fio2,creatinine,ph,hospitaldischargestatus,unitdischargestatus,ICUstay,SystolicBP,DiastolicBP,MAP
0,141168,1,Female,70,Caucasian,"Rhythm disturbance (atrial, supraventricular)",152.4,84.3,4,6,5,15,36.1,30.0,140,95.0,NaN,-1.0,2.30,-1.000,Expired,Expired,2.497222,92.707317,51.000000,64.902439
1,141178,1,Female,52,Caucasian,,162.6,54.4,-1,-1,-1,-3,36.4,16.0,88,-1.0,NaN,-1.0,-1.00,-1.000,Alive,Alive,0.015278,NaN,NaN,NaN
2,141194,1,Male,68,Caucasian,"Sepsis, renal/UTI (including bladder)",180.3,73.9,3,6,4,13,39.3,36.0,118,168.0,NaN,-1.0,2.51,-1.000,Alive,Alive,3.884028,92.828488,50.636628,64.700581
3,141197,1,Male,71,Caucasian,"Sepsis, pulmonary",162.6,102.1,4,6,5,15,-1.0,25.0,104,-1.0,NaN,-1.0,-1.00,-1.000,Alive,Alive,0.068750,NaN,NaN,NaN
4,141203,1,Female,77,Caucasian,"Arrest, respiratory (without cardiac arrest)",160.0,70.2,1,3,1,5,35.1,33.0,120,145.0,NaN,100.0,0.56,7.450,Alive,Alive,2.225694,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171172,3353235,1,Male,50,Caucasian,"CHF, congestive heart failure",175.3,90.0,4,6,5,15,36.2,50.0,106,-1.0,NaN,-1.0,-1.00,-1.000,Alive,Alive,0.765972,NaN,NaN,NaN
171173,3353237,1,Female,79,Caucasian,"Embolus, pulmonary",162.6,78.4,4,6,4,14,36.4,42.0,112,139.0,NaN,-1.0,1.07,-1.000,Alive,Alive,0.890972,NaN,NaN,NaN
171174,3353251,1,Male,73,African American,Cardiac arrest (with or without respiratory ar...,177.8,102.0,1,1,1,3,32.9,34.0,104,346.0,93.0,100.0,2.43,7.224,Alive,Alive,11.434028,131.617030,57.336898,82.096942
171175,3353254,1,Male,81,Caucasian,"Bleeding, lower GI",185.4,83.9,4,6,5,15,35.5,31.0,83,137.0,90.0,-1.0,-1.00,-1.000,Alive,Alive,0.487500,NaN,NaN,NaN


In [41]:
rawdf

,patientunitstayid,unitvisitnumber,gender,age,ethnicity,apacheadmissiondx,admissionheight,admissionweight,eyes,motor,verbal,GCSTotal,temperature,respiratoryrate,heartrate,glucose,O2,fio2,creatinine,ph,hospitaldischargestatus,unitdischargestatus,ICUstay
0,141168,1,Female,70,Caucasian,"Rhythm disturbance (atrial, supraventricular)",152.4,84.3,4,6,5,15,36.1,30.0,140,95.0,NaN,-1.0,2.30,-1.000,Expired,Expired,2.497222
1,141178,1,Female,52,Caucasian,,162.6,54.4,-1,-1,-1,-3,36.4,16.0,88,-1.0,NaN,-1.0,-1.00,-1.000,Alive,Alive,0.015278
2,141194,1,Male,68,Caucasian,"Sepsis, renal/UTI (including bladder)",180.3,73.9,3,6,4,13,39.3,36.0,118,168.0,NaN,-1.0,2.51,-1.000,Alive,Alive,3.884028
3,141197,1,Male,71,Caucasian,"Sepsis, pulmonary",162.6,102.1,4,6,5,15,-1.0,25.0,104,-1.0,NaN,-1.0,-1.00,-1.000,Alive,Alive,0.068750
4,141203,1,Female,77,Caucasian,"Arrest, respiratory (without cardiac arrest)",160.0,70.2,1,3,1,5,35.1,33.0,120,145.0,NaN,100.0,0.56,7.450,Alive,Alive,2.225694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171172,3353235,1,Male,50,Caucasian,"CHF, congestive heart failure",175.3,90.0,4,6,5,15,36.2,50.0,106,-1.0,NaN,-1.0,-1.00,-1.000,Alive,Alive,0.765972
171173,3353237,1,Female,79,Caucasian,"Embolus, pulmonary",162.6,78.4,4,6,4,14,36.4,42.0,112,139.0,NaN,-1.0,1.07,-1.000,Alive,Alive,0.890972
171174,3353251,1,Male,73,African American,Cardiac arrest (with or without respiratory ar...,177.8,102.0,1,1,1,3,32.9,34.0,104,346.0,93.0,100.0,2.43,7.224,Alive,Alive,11.434028
171175,3353254,1,Male,81,Caucasian,"Bleeding, lower GI",185.4,83.9,4,6,5,15,35.5,31.0,83,137.0,90.0,-1.0,-1.00,-1.000,Alive,Alive,0.487500


## Finding number of missing or null values -

In [50]:
rawdf1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 171177 entries, 0 to 171176
Data columns (total 26 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   patientunitstayid        171177 non-null  int64  
 1   unitvisitnumber          171177 non-null  int64  
 2   gender                   171177 non-null  object 
 3   age                      171177 non-null  object 
 4   ethnicity                171177 non-null  object 
 5   apacheadmissiondx        171177 non-null  object 
 6   admissionheight          167853 non-null  float64
 7   admissionweight          164555 non-null  float64
 8   eyes                     171177 non-null  int64  
 9   motor                    171177 non-null  int64  
 10  verbal                   171177 non-null  int64  
 11  GCSTotal                 171177 non-null  int64  
 12  temperature              171177 non-null  float64
 13  respiratoryrate          171177 non-null  float64
 14  hear

## Statistics of the features for preprocessing - 

In [7]:
rawdf.describe(include='all')

,patientunitstayid,unitvisitnumber,gender,age,ethnicity,apacheadmissiondx,admissionheight,admissionweight,temperature,respiratoryrate,heartrate,glucose,fio2,creatinine,ph,hospitaldischargestatus,unitdischargestatus,ICUstay
count,1.711770e+05,171177.000000,171177,171177,171177,171177,167853.000000,164555.000000,171177.000000,171177.000000,171177.000000,171177.000000,171177.000000,171177.000000,171177.000000,171177,171177,171177.000000
unique,NaN,NaN,5,90,7,393,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3,NaN
top,NaN,NaN,Male,> 89,Caucasian,"Sepsis, pulmonary",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alive,Alive,NaN
freq,NaN,NaN,92468,6084,131319,8427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,153532,161148,NaN
mean,1.747701e+06,1.151048,NaN,NaN,NaN,NaN,169.266201,83.850422,33.728719,24.784735,98.667993,141.184455,12.794373,0.999311,0.906831,NaN,NaN,4.254612
std,9.988850e+05,0.508644,NaN,NaN,NaN,NaN,13.845103,27.082294,9.725657,15.301895,33.263814,109.723423,28.329034,1.852455,3.506569,NaN,NaN,6.893154
min,1.411680e+05,1.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,NaN,NaN,-0.210417
25%,9.555340e+05,1.000000,NaN,NaN,NaN,NaN,162.500000,66.000000,36.100000,10.000000,84.000000,85.000000,-1.000000,0.470000,-1.000000,NaN,NaN,1.209028
50%,1.670376e+06,1.000000,NaN,NaN,NaN,NaN,170.000000,79.900000,36.400000,27.000000,104.000000,115.000000,-1.000000,0.810000,-1.000000,NaN,NaN,2.236111
75%,2.743419e+06,1.000000,NaN,NaN,NaN,NaN,177.800000,96.800000,36.700000,35.000000,120.000000,187.000000,-1.000000,1.360000,-1.000000,NaN,NaN,4.772222


## Impute missing values for Admission height and Admission weight 

In [0]:
rawdf['admissionheight']=rawdf.admissionheight.fillna(rawdf.admissionheight.median())
rawdf['admissionweight']=rawdf.admissionweight.fillna(rawdf.admissionweight.median())

In [9]:
rawdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171177 entries, 0 to 171176
Data columns (total 18 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   patientunitstayid        171177 non-null  int64  
 1   unitvisitnumber          171177 non-null  int64  
 2   gender                   171177 non-null  object 
 3   age                      171177 non-null  object 
 4   ethnicity                171177 non-null  object 
 5   apacheadmissiondx        171177 non-null  object 
 6   admissionheight          171177 non-null  float64
 7   admissionweight          171177 non-null  float64
 8   temperature              171177 non-null  float64
 9   respiratoryrate          171177 non-null  float64
 10  heartrate                171177 non-null  int64  
 11  glucose                  171177 non-null  float64
 12  fio2                     171177 non-null  float64
 13  creatinine               171177 non-null  float64
 14  ph  

## Convert age values to numeric

In [0]:
rawdf['age'].replace({'> 89': '90'}, inplace=True)
rawdf['age']=rawdf['age'].apply(pd.to_numeric)

In [11]:
rawdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171177 entries, 0 to 171176
Data columns (total 18 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   patientunitstayid        171177 non-null  int64  
 1   unitvisitnumber          171177 non-null  int64  
 2   gender                   171177 non-null  object 
 3   age                      171109 non-null  float64
 4   ethnicity                171177 non-null  object 
 5   apacheadmissiondx        171177 non-null  object 
 6   admissionheight          171177 non-null  float64
 7   admissionweight          171177 non-null  float64
 8   temperature              171177 non-null  float64
 9   respiratoryrate          171177 non-null  float64
 10  heartrate                171177 non-null  int64  
 11  glucose                  171177 non-null  float64
 12  fio2                     171177 non-null  float64
 13  creatinine               171177 non-null  float64
 14  ph  

## Age has some null values. Removing unknown/other category in gender and ethnicity. : 

In [0]:
missing = rawdf[(rawdf['gender'] == '') |(rawdf['gender'] == 'Other')|(rawdf['gender'] == 'Unknown')| (rawdf['ethnicity'] == '')|(rawdf['unitdischargestatus']=='')|(rawdf['hospitaldischargestatus']=='')].index
rawdf = rawdf.drop(index=missing)
rawdf = rawdf.dropna()

## Checking if we have anymore more missing or ambiguous values for features

In [13]:
rawdf.isnull().sum()

patientunitstayid          0
unitvisitnumber            0
gender                     0
age                        0
ethnicity                  0
apacheadmissiondx          0
admissionheight            0
admissionweight            0
temperature                0
respiratoryrate            0
heartrate                  0
glucose                    0
fio2                       0
creatinine                 0
ph                         0
hospitaldischargestatus    0
unitdischargestatus        0
ICUstay                    0
dtype: int64

In [14]:
rawdf

,patientunitstayid,unitvisitnumber,gender,age,ethnicity,apacheadmissiondx,admissionheight,admissionweight,temperature,respiratoryrate,heartrate,glucose,fio2,creatinine,ph,hospitaldischargestatus,unitdischargestatus,ICUstay
0,224606,1,Female,76.0,Caucasian,"GI perforation/rupture, surgery for",160.0,79.9,36.7,38.0,119,81.0,100.0,1.10,7.280,Alive,Alive,3.031250
1,204602,1,Female,29.0,Caucasian,"Cardiovascular medical, other",162.6,88.5,36.5,55.0,138,82.0,-1.0,3.21,-1.000,Alive,Alive,4.201389
2,157016,1,Female,23.0,Caucasian,"GI medical, other",162.6,63.5,36.4,45.0,115,82.0,-1.0,0.61,-1.000,Alive,Alive,5.210417
3,211144,1,Female,67.0,Caucasian,Aortic valve replacement (isolated),160.0,86.2,36.8,33.0,104,85.0,50.0,0.80,7.380,Alive,Alive,3.893056
4,221005,1,Female,68.0,Caucasian,"Complications of previous open-heart surgery, ...",162.6,88.0,36.4,43.0,87,37.0,32.0,-1.00,7.340,Alive,Alive,2.803472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171172,3351785,1,Male,74.0,Caucasian,Head only trauma,182.9,75.7,36.8,49.0,50,-1.0,-1.0,-1.00,-1.000,Alive,Alive,1.790278
171173,3340321,1,Male,44.0,Caucasian,"Infarction, acute myocardial (MI)",185.4,130.6,36.7,45.0,102,220.0,-1.0,0.91,-1.000,Alive,Alive,2.689583
171174,3345874,1,Male,55.0,African American,Rhythm disturbance (conduction defect),190.5,165.9,32.2,37.0,90,254.0,-1.0,2.30,-1.000,Alive,Alive,1.968056
171175,3352628,1,Male,60.0,African American,"Apnea-sleep; surgery for (i.e., UPPP - uvulopa...",170.1,120.2,36.3,37.0,114,102.0,50.0,1.00,7.363,Alive,Alive,1.481250


## Exclusion criteria for mortality prediction
            age <= 18 and age >= 89
            more than 1 ICU stay
            los < 48h

In [15]:
excl = rawdf[(rawdf['age']<= 18) | (rawdf['age']>=89) |(rawdf['unitvisitnumber']>1)| (rawdf['ICUstay']<2)].index
rawdf = rawdf.drop(index=excl)
rawdf

,patientunitstayid,unitvisitnumber,gender,age,ethnicity,apacheadmissiondx,admissionheight,admissionweight,temperature,respiratoryrate,heartrate,glucose,fio2,creatinine,ph,hospitaldischargestatus,unitdischargestatus,ICUstay
0,224606,1,Female,76.0,Caucasian,"GI perforation/rupture, surgery for",160.0,79.9,36.7,38.0,119,81.0,100.0,1.10,7.280,Alive,Alive,3.031250
1,204602,1,Female,29.0,Caucasian,"Cardiovascular medical, other",162.6,88.5,36.5,55.0,138,82.0,-1.0,3.21,-1.000,Alive,Alive,4.201389
2,157016,1,Female,23.0,Caucasian,"GI medical, other",162.6,63.5,36.4,45.0,115,82.0,-1.0,0.61,-1.000,Alive,Alive,5.210417
3,211144,1,Female,67.0,Caucasian,Aortic valve replacement (isolated),160.0,86.2,36.8,33.0,104,85.0,50.0,0.80,7.380,Alive,Alive,3.893056
4,221005,1,Female,68.0,Caucasian,"Complications of previous open-heart surgery, ...",162.6,88.0,36.4,43.0,87,37.0,32.0,-1.00,7.340,Alive,Alive,2.803472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171169,3351729,1,Male,64.0,African American,"CABG alone, coronary artery bypass grafting",182.9,112.5,36.1,41.0,91,82.0,100.0,1.19,7.356,Alive,Alive,6.891667
171170,3340416,1,Male,64.0,Caucasian,"CABG alone, coronary artery bypass grafting",193.0,109.1,35.8,5.0,91,181.0,100.0,0.60,7.315,Alive,Alive,3.617361
171171,3329614,1,Male,71.0,African American,"Hemorrhage/hematoma, intracranial",188.0,90.7,36.7,50.0,109,97.0,-1.0,2.52,-1.000,Expired,Alive,3.704167
171173,3340321,1,Male,44.0,Caucasian,"Infarction, acute myocardial (MI)",185.4,130.6,36.7,45.0,102,220.0,-1.0,0.91,-1.000,Alive,Alive,2.689583


In [16]:
rawdf.describe(include='all')

,patientunitstayid,unitvisitnumber,gender,age,ethnicity,apacheadmissiondx,admissionheight,admissionweight,temperature,respiratoryrate,heartrate,glucose,fio2,creatinine,ph,hospitaldischargestatus,unitdischargestatus,ICUstay
count,7.385400e+04,73854.0,73854,73854.000000,73854,73854,73854.000000,73854.000000,73854.000000,73854.000000,73854.000000,73854.000000,73854.000000,73854.000000,73854.000000,73854,73854,73854.000000
unique,NaN,NaN,2,NaN,6,390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,NaN
top,NaN,NaN,Male,NaN,Caucasian,"Sepsis, pulmonary",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alive,Alive,NaN
freq,NaN,NaN,40549,NaN,56581,4263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66024,69194,NaN
mean,1.777731e+06,1.0,NaN,63.238498,NaN,NaN,169.456453,85.033961,34.497731,25.704222,103.269979,151.923238,18.938525,1.221629,1.707134,NaN,NaN,5.976045
std,1.005705e+06,0.0,NaN,15.538382,NaN,NaN,13.073183,27.349791,8.294009,15.223788,31.333620,107.741193,32.376026,1.914339,3.912000,NaN,NaN,7.474907
min,1.411680e+05,1.0,NaN,19.000000,NaN,NaN,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,NaN,NaN,2.000000
25%,9.752198e+05,1.0,NaN,54.000000,NaN,NaN,162.600000,67.500000,36.000000,11.000000,91.000000,91.000000,-1.000000,0.580000,-1.000000,NaN,NaN,2.772917
50%,1.660696e+06,1.0,NaN,65.000000,NaN,NaN,170.100000,80.450000,36.400000,28.000000,107.000000,130.000000,-1.000000,0.900000,-1.000000,NaN,NaN,4.003472
75%,2.804134e+06,1.0,NaN,75.000000,NaN,NaN,177.800000,97.900000,36.700000,36.000000,123.000000,196.000000,40.000000,1.540000,7.300000,NaN,NaN,6.841493


## Encoding the categorical values
### Adding columns having the encoded values for ethnicity, admission diagnosis, gender, hospital and ICU discharge status

In [0]:
lb_make = LabelEncoder()
rawdf['ethnicity'] =rawdf['ethnicity'].astype('category')
rawdf['gender'] =rawdf['gender'].astype('category')
rawdf['hospitaldischargestatus'] =rawdf['hospitaldischargestatus'].astype('category')
rawdf['unitdischargestatus'] =rawdf['unitdischargestatus'].astype('category')
rawdf['apacheadmissiondx'] =rawdf['apacheadmissiondx'].astype('category')


In [18]:
rawdf['ethnicity_code'] = lb_make.fit_transform(rawdf['ethnicity'])
rawdf['diag_code'] = lb_make.fit_transform(rawdf['apacheadmissiondx'])
rawdf['gender_code'] = rawdf['gender'].map({"Male":0,"Female":1})
rawdf['hosp_alive'] = rawdf['hospitaldischargestatus'].map({"Expired":0,"Alive":1})
rawdf['icu_alive'] = rawdf['unitdischargestatus'].map({"Expired":0,"Alive":1})
rawdf

,patientunitstayid,unitvisitnumber,gender,age,ethnicity,apacheadmissiondx,admissionheight,admissionweight,temperature,respiratoryrate,heartrate,glucose,fio2,creatinine,ph,hospitaldischargestatus,unitdischargestatus,ICUstay,ethnicity_code,diag_code,gender_code,hosp_alive,icu_alive
0,224606,1,Female,76.0,Caucasian,"GI perforation/rupture, surgery for",160.0,79.9,36.7,38.0,119,81.0,100.0,1.10,7.280,Alive,Alive,3.031250,2,172,1,1,1
1,204602,1,Female,29.0,Caucasian,"Cardiovascular medical, other",162.6,88.5,36.5,55.0,138,82.0,-1.0,3.21,-1.000,Alive,Alive,4.201389,2,92,1,1,1
2,157016,1,Female,23.0,Caucasian,"GI medical, other",162.6,63.5,36.4,45.0,115,82.0,-1.0,0.61,-1.000,Alive,Alive,5.210417,2,168,1,1,1
3,211144,1,Female,67.0,Caucasian,Aortic valve replacement (isolated),160.0,86.2,36.8,33.0,104,85.0,50.0,0.80,7.380,Alive,Alive,3.893056,2,34,1,1,1
4,221005,1,Female,68.0,Caucasian,"Complications of previous open-heart surgery, ...",162.6,88.0,36.4,43.0,87,37.0,32.0,-1.00,7.340,Alive,Alive,2.803472,2,119,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171169,3351729,1,Male,64.0,African American,"CABG alone, coronary artery bypass grafting",182.9,112.5,36.1,41.0,91,82.0,100.0,1.19,7.356,Alive,Alive,6.891667,0,60,0,1,1
171170,3340416,1,Male,64.0,Caucasian,"CABG alone, coronary artery bypass grafting",193.0,109.1,35.8,5.0,91,181.0,100.0,0.60,7.315,Alive,Alive,3.617361,2,60,0,1,1
171171,3329614,1,Male,71.0,African American,"Hemorrhage/hematoma, intracranial",188.0,90.7,36.7,50.0,109,97.0,-1.0,2.52,-1.000,Expired,Alive,3.704167,0,209,0,0,1
171173,3340321,1,Male,44.0,Caucasian,"Infarction, acute myocardial (MI)",185.4,130.6,36.7,45.0,102,220.0,-1.0,0.91,-1.000,Alive,Alive,2.689583,2,229,0,1,1


## Handling outliers
### There are outliers in columns : admissionheight,admissionweight
###  Maximum values are way beyond 90th percentile

In [19]:
rawdf.describe()

,patientunitstayid,unitvisitnumber,age,admissionheight,admissionweight,temperature,respiratoryrate,heartrate,glucose,fio2,creatinine,ph,ICUstay,ethnicity_code,diag_code
count,7.385400e+04,73854.0,73854.000000,73854.000000,73854.000000,73854.000000,73854.000000,73854.000000,73854.000000,73854.000000,73854.000000,73854.000000,73854.000000,73854.000000,73854.000000
mean,1.777731e+06,1.0,63.238498,169.456453,85.033961,34.497731,25.704222,103.269979,151.923238,18.938525,1.221629,1.707134,5.976045,1.935535,192.355539
std,1.005705e+06,0.0,15.538382,13.073183,27.349791,8.294009,15.223788,31.333620,107.741193,32.376026,1.914339,3.912000,7.474907,1.003049,116.360892
min,1.411680e+05,1.0,19.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2.000000,0.000000,0.000000
25%,9.752198e+05,1.0,54.000000,162.600000,67.500000,36.000000,11.000000,91.000000,91.000000,-1.000000,0.580000,-1.000000,2.772917,2.000000,73.000000
50%,1.660696e+06,1.0,65.000000,170.100000,80.450000,36.400000,28.000000,107.000000,130.000000,-1.000000,0.900000,-1.000000,4.003472,2.000000,189.000000
75%,2.804134e+06,1.0,75.000000,177.800000,97.900000,36.700000,36.000000,123.000000,196.000000,40.000000,1.540000,7.300000,6.841493,2.000000,322.000000
max,3.353251e+06,1.0,88.000000,612.600000,953.000000,42.300000,60.000000,220.000000,1791.000000,100.000000,24.600000,7.810000,749.036111,5.000000,389.000000


### Quantile based flooring and capping for height, weight, temperature, respiratory rate, heartrate,glucose

In [0]:
rawdf['admissionheight'] = np.where(rawdf['admissionheight']<rawdf['admissionheight'].quantile(0.10),rawdf['admissionheight'].quantile(0.10),rawdf['admissionheight'])
rawdf['admissionheight'] = np.where(rawdf['admissionheight']>rawdf['admissionheight'].quantile(0.90),rawdf['admissionheight'].quantile(0.90),rawdf['admissionheight'])
rawdf['admissionweight'] = np.where(rawdf['admissionweight']<rawdf['admissionweight'].quantile(0.10),rawdf['admissionweight'].quantile(0.10),rawdf['admissionweight'])
rawdf['admissionweight'] = np.where(rawdf['admissionweight']>rawdf['admissionweight'].quantile(0.90),rawdf['admissionweight'].quantile(0.90),rawdf['admissionweight'])
rawdf['temperature'] = np.where(rawdf['temperature']<rawdf['temperature'].quantile(0.10),rawdf['temperature'].quantile(0.10),rawdf['temperature'])
rawdf['respiratoryrate'] = np.where(rawdf['respiratoryrate']<rawdf['respiratoryrate'].quantile(0.10),rawdf['respiratoryrate'].quantile(0.10),rawdf['respiratoryrate'])
rawdf['heartrate'] = np.where(rawdf['heartrate']<rawdf['heartrate'].quantile(0.10),rawdf['heartrate'].quantile(0.10),rawdf['heartrate'])
rawdf['glucose'] = np.where(rawdf['glucose']<rawdf['glucose'].quantile(0.10),rawdf['glucose'].quantile(0.10),rawdf['glucose'])
rawdf['glucose'] = np.where(rawdf['glucose']>rawdf['glucose'].quantile(0.90),rawdf['glucose'].quantile(0.90),rawdf['glucose'])

### fio2, creatinine and ph have a lot of values (Around 40,000) that are -1, hence we will just trim them from the dataset.

In [21]:
index = rawdf[(rawdf['fio2']==-1)|(rawdf['creatinine']==-1)|(rawdf['ph']==-1)].index
#index = rawdf[(rawdf['ph']==-1)].index
rawdf.drop(index=index,inplace=True)
rawdf

,patientunitstayid,unitvisitnumber,gender,age,ethnicity,apacheadmissiondx,admissionheight,admissionweight,temperature,respiratoryrate,heartrate,glucose,fio2,creatinine,ph,hospitaldischargestatus,unitdischargestatus,ICUstay,ethnicity_code,diag_code,gender_code,hosp_alive,icu_alive
0,224606,1,Female,76.0,Caucasian,"GI perforation/rupture, surgery for",160.0,79.9,36.7,38.0,119.0,81.0,100.0,1.10,7.280,Alive,Alive,3.031250,2,172,1,1,1
3,211144,1,Female,67.0,Caucasian,Aortic valve replacement (isolated),160.0,86.2,36.8,33.0,104.0,85.0,50.0,0.80,7.380,Alive,Alive,3.893056,2,34,1,1,1
5,151900,1,Female,66.0,Caucasian,"Sepsis, pulmonary",165.1,86.8,36.5,10.0,120.0,90.0,40.0,0.84,7.420,Alive,Alive,2.418750,2,333,1,1,1
11,151179,1,Female,59.0,Caucasian,"Sepsis, cutaneous/soft tissue",155.0,79.9,35.1,52.0,171.0,58.0,28.0,3.02,7.230,Expired,Alive,3.406250,2,330,1,0,1
31,169525,1,Female,59.0,Caucasian,"Endarterectomy, carotid",162.6,74.1,36.7,34.0,104.0,209.0,32.0,0.52,7.330,Alive,Alive,2.020139,2,148,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171160,3340781,1,Male,79.0,Caucasian,"CABG alone, coronary artery bypass grafting",182.9,92.1,36.0,43.0,76.0,165.0,100.0,1.15,7.397,Alive,Alive,2.554167,2,60,0,1,1
171168,3329043,1,Male,67.0,Caucasian,"CHF, congestive heart failure",172.7,117.4,36.7,48.0,139.0,160.0,50.0,1.90,7.374,Alive,Alive,11.320833,2,72,0,1,1
171169,3351729,1,Male,64.0,African American,"CABG alone, coronary artery bypass grafting",182.9,112.5,36.1,41.0,91.0,82.0,100.0,1.19,7.356,Alive,Alive,6.891667,0,60,0,1,1
171170,3340416,1,Male,64.0,Caucasian,"CABG alone, coronary artery bypass grafting",182.9,109.1,35.8,7.0,91.0,181.0,100.0,0.60,7.315,Alive,Alive,3.617361,2,60,0,1,1


In [22]:
rawdf.describe()

,patientunitstayid,unitvisitnumber,age,admissionheight,admissionweight,temperature,respiratoryrate,heartrate,glucose,fio2,creatinine,ph,ICUstay,ethnicity_code,diag_code
count,2.163300e+04,21633.0,21633.000000,21633.000000,21633.000000,21633.000000,21633.000000,21633.000000,21633.000000,21633.000000,21633.000000,21633.000000,21633.000000,21633.000000,21633.000000
mean,1.737257e+06,1.0,63.338002,169.851067,85.052895,36.346302,26.477317,107.543244,161.586511,60.859529,1.675962,7.359205,7.202326,1.949938,181.026718
std,1.064091e+06,0.0,14.961131,9.193929,19.731633,0.937168,15.245038,29.127377,68.134189,26.054000,1.680654,0.096211,8.367349,1.013563,117.387159
min,1.412030e+05,1.0,19.000000,155.000000,56.400000,35.100000,7.000000,55.000000,55.000000,21.000000,0.100000,6.531000,2.000000,0.000000,0.000000
25%,8.198420e+05,1.0,55.000000,162.600000,68.700000,35.800000,10.000000,94.000000,98.000000,40.000000,0.760000,7.310000,3.147917,2.000000,72.000000
50%,1.576735e+06,1.0,65.000000,170.200000,82.500000,36.400000,29.000000,110.000000,161.000000,50.000000,1.140000,7.367000,5.012500,2.000000,146.000000
75%,2.914108e+06,1.0,75.000000,177.800000,100.200000,36.700000,37.000000,126.000000,213.000000,90.000000,1.870000,7.420000,8.725694,2.000000,316.000000
max,3.353251e+06,1.0,88.000000,182.900000,117.400000,42.300000,60.000000,220.000000,273.000000,100.000000,21.500000,7.810000,749.036111,5.000000,389.000000


In [23]:
print(f'90th Quantile for ICU stay - {rawdf.ICUstay.quantile(0.90)}')
print(f'90th Quantile for creatinine - {rawdf.creatinine.quantile(0.90)}')

90th Quantile for ICU stay - 14.44694444444444
90th Quantile for creatinine - 3.37


### Fixing distribution of creatinine and ICU stay (notice the max value)

In [24]:
ind = rawdf[(rawdf['creatinine']> rawdf['creatinine'].quantile(0.90))|(rawdf['ICUstay'] > rawdf['ICUstay'].quantile(0.90))].index
rawdf.drop(index=ind,inplace=True)
rawdf

,patientunitstayid,unitvisitnumber,gender,age,ethnicity,apacheadmissiondx,admissionheight,admissionweight,temperature,respiratoryrate,heartrate,glucose,fio2,creatinine,ph,hospitaldischargestatus,unitdischargestatus,ICUstay,ethnicity_code,diag_code,gender_code,hosp_alive,icu_alive
0,224606,1,Female,76.0,Caucasian,"GI perforation/rupture, surgery for",160.0,79.9,36.7,38.0,119.0,81.0,100.0,1.10,7.280,Alive,Alive,3.031250,2,172,1,1,1
3,211144,1,Female,67.0,Caucasian,Aortic valve replacement (isolated),160.0,86.2,36.8,33.0,104.0,85.0,50.0,0.80,7.380,Alive,Alive,3.893056,2,34,1,1,1
5,151900,1,Female,66.0,Caucasian,"Sepsis, pulmonary",165.1,86.8,36.5,10.0,120.0,90.0,40.0,0.84,7.420,Alive,Alive,2.418750,2,333,1,1,1
11,151179,1,Female,59.0,Caucasian,"Sepsis, cutaneous/soft tissue",155.0,79.9,35.1,52.0,171.0,58.0,28.0,3.02,7.230,Expired,Alive,3.406250,2,330,1,0,1
31,169525,1,Female,59.0,Caucasian,"Endarterectomy, carotid",162.6,74.1,36.7,34.0,104.0,209.0,32.0,0.52,7.330,Alive,Alive,2.020139,2,148,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171160,3340781,1,Male,79.0,Caucasian,"CABG alone, coronary artery bypass grafting",182.9,92.1,36.0,43.0,76.0,165.0,100.0,1.15,7.397,Alive,Alive,2.554167,2,60,0,1,1
171168,3329043,1,Male,67.0,Caucasian,"CHF, congestive heart failure",172.7,117.4,36.7,48.0,139.0,160.0,50.0,1.90,7.374,Alive,Alive,11.320833,2,72,0,1,1
171169,3351729,1,Male,64.0,African American,"CABG alone, coronary artery bypass grafting",182.9,112.5,36.1,41.0,91.0,82.0,100.0,1.19,7.356,Alive,Alive,6.891667,0,60,0,1,1
171170,3340416,1,Male,64.0,Caucasian,"CABG alone, coronary artery bypass grafting",182.9,109.1,35.8,7.0,91.0,181.0,100.0,0.60,7.315,Alive,Alive,3.617361,2,60,0,1,1


In [25]:
rawdf.describe()

,patientunitstayid,unitvisitnumber,age,admissionheight,admissionweight,temperature,respiratoryrate,heartrate,glucose,fio2,creatinine,ph,ICUstay,ethnicity_code,diag_code
count,1.756100e+04,17561.0,17561.000000,17561.000000,17561.000000,17561.000000,17561.000000,17561.000000,17561.000000,17561.000000,17561.000000,17561.000000,17561.000000,17561.000000,17561.000000
mean,1.739771e+06,1.0,63.697625,169.786734,84.819502,36.344335,26.126143,106.972040,160.933660,60.147139,1.220194,7.363211,5.451854,1.965150,175.593189
std,1.061213e+06,0.0,14.980919,9.178901,19.596382,0.913762,15.211334,28.874482,66.840073,25.804450,0.660309,0.091620,3.047363,0.968369,116.286294
min,1.412030e+05,1.0,19.000000,155.000000,56.400000,35.100000,7.000000,55.000000,55.000000,21.000000,0.100000,6.531000,2.000000,0.000000,0.000000
25%,8.235880e+05,1.0,55.000000,162.600000,68.500000,35.800000,10.000000,93.000000,99.000000,40.000000,0.730000,7.311000,3.018056,2.000000,64.000000
50%,1.577869e+06,1.0,66.000000,170.100000,82.300000,36.400000,28.000000,109.000000,160.000000,50.000000,1.040000,7.370000,4.524306,2.000000,143.000000
75%,2.908853e+06,1.0,75.000000,177.800000,100.000000,36.700000,37.000000,125.000000,210.000000,80.000000,1.540000,7.420000,7.108333,2.000000,304.000000
max,3.353251e+06,1.0,88.000000,182.900000,117.400000,42.300000,60.000000,220.000000,273.000000,100.000000,3.370000,7.700000,14.443056,5.000000,389.000000


### Dropping columns that are not required

In [26]:
rawdf = rawdf.drop(['unitvisitnumber','patientunitstayid','apacheadmissiondx','ethnicity','gender','unitdischargestatus','hospitaldischargestatus'],axis=1)
rawdf

,age,admissionheight,admissionweight,temperature,respiratoryrate,heartrate,glucose,fio2,creatinine,ph,ICUstay,ethnicity_code,diag_code,gender_code,hosp_alive,icu_alive
0,76.0,160.0,79.9,36.7,38.0,119.0,81.0,100.0,1.10,7.280,3.031250,2,172,1,1,1
3,67.0,160.0,86.2,36.8,33.0,104.0,85.0,50.0,0.80,7.380,3.893056,2,34,1,1,1
5,66.0,165.1,86.8,36.5,10.0,120.0,90.0,40.0,0.84,7.420,2.418750,2,333,1,1,1
11,59.0,155.0,79.9,35.1,52.0,171.0,58.0,28.0,3.02,7.230,3.406250,2,330,1,0,1
31,59.0,162.6,74.1,36.7,34.0,104.0,209.0,32.0,0.52,7.330,2.020139,2,148,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171160,79.0,182.9,92.1,36.0,43.0,76.0,165.0,100.0,1.15,7.397,2.554167,2,60,0,1,1
171168,67.0,172.7,117.4,36.7,48.0,139.0,160.0,50.0,1.90,7.374,11.320833,2,72,0,1,1
171169,64.0,182.9,112.5,36.1,41.0,91.0,82.0,100.0,1.19,7.356,6.891667,0,60,0,1,1
171170,64.0,182.9,109.1,35.8,7.0,91.0,181.0,100.0,0.60,7.315,3.617361,2,60,0,1,1


In [27]:
rawdf = rawdf.rename(columns={'admissionheight':'height','admissionweight':'weight','ethnicity_code':'ethnicity','gender_code':'gender','ICUstay':'los'})
rawdf

,age,height,weight,temperature,respiratoryrate,heartrate,glucose,fio2,creatinine,ph,los,ethnicity,diag_code,gender,hosp_alive,icu_alive
0,76.0,160.0,79.9,36.7,38.0,119.0,81.0,100.0,1.10,7.280,3.031250,2,172,1,1,1
3,67.0,160.0,86.2,36.8,33.0,104.0,85.0,50.0,0.80,7.380,3.893056,2,34,1,1,1
5,66.0,165.1,86.8,36.5,10.0,120.0,90.0,40.0,0.84,7.420,2.418750,2,333,1,1,1
11,59.0,155.0,79.9,35.1,52.0,171.0,58.0,28.0,3.02,7.230,3.406250,2,330,1,0,1
31,59.0,162.6,74.1,36.7,34.0,104.0,209.0,32.0,0.52,7.330,2.020139,2,148,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171160,79.0,182.9,92.1,36.0,43.0,76.0,165.0,100.0,1.15,7.397,2.554167,2,60,0,1,1
171168,67.0,172.7,117.4,36.7,48.0,139.0,160.0,50.0,1.90,7.374,11.320833,2,72,0,1,1
171169,64.0,182.9,112.5,36.1,41.0,91.0,82.0,100.0,1.19,7.356,6.891667,0,60,0,1,1
171170,64.0,182.9,109.1,35.8,7.0,91.0,181.0,100.0,0.60,7.315,3.617361,2,60,0,1,1


## Converting this data to csv file

In [0]:
rawdf.to_csv('rawdata.csv',index=False,header=True)

In [0]:
from google.colab import files

In [0]:
files.download('rawdata.csv')